In [15]:
from __future__ import unicode_literals
import classifiers
import re
import nltk
from Politweet import get_tweets, get_transcript
import ratings
from sentiment import polarity_train, classify, prob_classify, plus_df, minus_df
import pandas as pd
pd.set_option('display.max_colwidth', 1200)

In [16]:
tweets = get_tweets("./datasets/tweets.tsv")
transcript = get_transcript("./datasets/transcript.csv")
todo = set(tweets.index)


In [17]:
tweets.to_pickle('./datasets/tweets.pickle')

In [18]:
polarity = polarity_train(tweets)
polarity_minus = set(i for i,_ in classify(minus_df(tweets), polarity))
polarity_plus = set(i for i,_ in classify(plus_df(tweets), polarity))
polarities = tweets.reindex(polarity_minus | polarity_plus).sort()
print len(polarities), "- polarity"
todo -= set(polarities.index)

482 - polarity


In [19]:
others = ratings.all(tweets, ratings.OTHER)
print len(others), "- marked as OTHERS"
todo -= set(others.index)

151 - marked as OTHERS


In [20]:
print len(todo), "- untagged"

2605 - untagged


In [21]:
left = tweets.reindex(todo).sort()

In [22]:
left[["content","tokens", "clean"]][:2]

,content,tokens,clean
tweet.id,,,
936466790,Jim Lehrer just directed the debate audience ... 30 seconds ... #tweetdebate,"[jim, lehrer, direct, debate, audience, second, tweetdebate]","[{u'lemma': u'jim', u'token': u'jim', u'pos': u'NN'}, {u'lemma': u'lehrer', u'token': u'lehrer', u'pos': u'NN'}, {u'lemma': u'direct', u'token': u'directed', u'pos': u'VBN'}, {u'lemma': u'debate', u'token': u'debate', u'pos': u'NN'}, {u'lemma': u'audience', u'token': u'audience', u'pos': u'NN'}, {u'lemma': u'', u'token': u'', u'pos': u''}, {u'lemma': u'30', u'token': u'30', u'pos': u'CD'}, {u'lemma': u'second', u'token': u'seconds', u'pos': u'NNS'}, {u'lemma': u'', u'token': u'', u'pos': u''}, {u'lemma': u'', u'token': u'', u'pos': u''}, {u'lemma': u'tweetdebate', u'token': u'tweetdebate', u'pos': u'NN'}]"
936466992,Here we go. #tweetdebate,[tweetdebate],"[{u'lemma': u'go', u'token': u'go', u'pos': u'VBP'}, {u'lemma': u'', u'token': u'', u'pos': u''}, {u'lemma': u'', u'token': u'', u'pos': u''}, {u'lemma': u'tweetdebate', u'token': u'tweetdebate', u'pos': u'NN'}]"


In [23]:
for i, tweet in tweets[:3].iterrows():
    print [token for token in tweet["tokens"]]
    
tokens = [" ".join(tweet["tokens"]) for i,tweet in tweets.iterrows()]
tokens[:1]

[u'watch', u'tweetdebate', u'drink', u'wait', u'start', u'cringing', u'mccain', u'blunder']
[u'ahg3', u'michdot', u'yeah', u'slime', u'actually', u'second', u'choice', u'say', u'first', u'one', u'okay', u'roll']
[u'prepare', u'heart', u'attack', u'tweetdebate']


[u'watch tweetdebate drink wait start cringing mccain blunder']

In [24]:
# from collections import Counter
# from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
# count = Counter([token for i, tweet in tweets.iterrows() for token in tweet["tokens"]])

# vectorizer = TfidfVectorizer(lowercase=False)
# tfidf_model = vectorizer.fit_transform(tokens)
# feature_names = vectorizer.get_feature_names()
# df = pd.DataFrame([
#         (feature_names[col], tfidf_model[0, col])
#         for col in tfidf_model.nonzero()[1]])
# df.sort(1, ascending=False)

In [25]:
# for col in tfidf_model.nonzero()[1]:
#     print tfidf_model[0, col]
#     print (feature_names[col], tfidf_model[0, col])

In [26]:
# tweets[tweets["content"].str.contains("zoom")][['content']]

In [87]:
import json
from sklearn.cross_validation import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from nltk import sent_tokenize, word_tokenize, FreqDist, WordNetLemmatizer
from nltk.corpus import stopwords
import classifiers

neg = [
    (t['clean'], 'neg')
    for i,t in ratings.all(tweets, ratings.NEGATIVE).iterrows()]

pos = [
    (t['clean'], 'pos')
    for i,t in ratings.all(tweets, ratings.POSITIVE).iterrows()]

train, test = train_test_split(
    pos + neg, 
    test_size = .2, 
    random_state = 10)

def featurize(tweet):
#     # tokenize into words
#     tokens = [word for sent in sent_tokenize(tweet) for word in word_tokenize(sent)]

#     # remove stopwords
#     stop = stopwords.words('english')
#     tokens = [token for token in tokens if token not in stop]

#     # remove words less than three letters
#     tokens = [word for word in tokens if len(word) >= 3]

#     # lower capitalization
#     tokens = [word.lower() for word in tokens]

#     # lemmatize
#     lmtzr = WordNetLemmatizer()
#     tokens = [lmtzr.lemmatize(word) for word in tokens]
    tokens = [token['lemma'] for token in tweet]
    return tokens

def run_classifier(train, test, vctrzr, clsfr):
    # vectorize with above featurize() function and also vectorize test set with training fit
    X_train = vctrzr.fit_transform([i[0] for i in train])
    X_test = vctrzr.transform([i[0] for i in test])

    # fit the classifier with training data
    clsfr.fit(X_train, [i[1] for i in train])
    
    # grab accuracy
    scr = clsfr.score(X_test, [i[1] for i in test])

    # grab important features
    imp_features = sorted(zip(clsfr.coef_[0], vctrzr.get_feature_names()))
    
    return scr, imp_features

In [88]:
vectorizer = CountVectorizer(tokenizer = featurize, binary = True, lowercase=False)
classifier = MultinomialNB()

score, imp_features = run_classifier(train, test, vectorizer, classifier)
score, imp_features[0:40], imp_features[-21:-1]

(0.78804347826086951,
 [(-8.3617082885758425, u'08'),
  (-8.3617082885758425, u'100'),
  (-8.3617082885758425, u'1000'),
  (-8.3617082885758425, u'10000'),
  (-8.3617082885758425, u'12'),
  (-8.3617082885758425, u'1950s'),
  (-8.3617082885758425, u'2004'),
  (-8.3617082885758425, u'2019'),
  (-8.3617082885758425, u'2030'),
  (-8.3617082885758425, u'2050'),
  (-8.3617082885758425, u'4'),
  (-8.3617082885758425, u'42'),
  (-8.3617082885758425, u'5000000'),
  (-8.3617082885758425, u'700b'),
  (-8.3617082885758425, u'72'),
  (-8.3617082885758425, u'8'),
  (-8.3617082885758425, u'9'),
  (-8.3617082885758425, u'911'),
  (-8.3617082885758425, u'915'),
  (-8.3617082885758425, u'abroad'),
  (-8.3617082885758425, u'abslutely'),
  (-8.3617082885758425, u'accommodate'),
  (-8.3617082885758425, u'accord'),
  (-8.3617082885758425, u'accuse'),
  (-8.3617082885758425, u'accusing'),
  (-8.3617082885758425, u'act'),
  (-8.3617082885758425, u'administration'),
  (-8.3617082885758425, u'adult'),
  (-8.361

In [89]:
vectorizer = TfidfVectorizer(tokenizer = featurize, lowercase=False)
classifier = LinearSVC()

score, imp_features = run_classifier(train, test, vectorizer, classifier)
score, imp_features[0:20]
score, imp_features[0:20], imp_features[-21:-1]

(0.80434782608695654,
 [(-1.4759941272554509, u'mccain'),
  (-1.3825646012153336, u'nt'),
  (-1.3338605212433292, u'stop'),
  (-1.0532868309789634, u'false'),
  (-1.044511571148947, u'yet'),
  (-0.99757736410358078, u'bracelet'),
  (-0.98357832813971879, u'miss'),
  (-0.93125886610797315, u'different'),
  (-0.91165056309537285, u'would'),
  (-0.87604562897801563, u'mention'),
  (-0.87475820281007299, u'much'),
  (-0.87175753392485311, u'claim'),
  (-0.82179803895363868, u'republican'),
  (-0.80472382441829204, u'liar'),
  (-0.78210207161398437, u'shit'),
  (-0.76744079592402981, u'peacekeeper'),
  (-0.7625471000411802, u'talk'),
  (-0.75688883555742281, u'cheap'),
  (-0.7452562632388392, u'lose'),
  (-0.74291864696840226, u'hear')],
 [(0.91130887766322966, u'bring'),
  (0.91606200882196787, u'father'),
  (0.95427349168821629, u'debate'),
  (0.95903771017843642, u'easily'),
  (0.96394932814971468, u'clear'),
  (0.98987629408193556, u'hold'),
  (0.99780119083797147, u'blast'),
  (1.00976